У нас есть текст:

In a hole in the ground there lived a hobbit. Not a nasty, dirty, wet hole, filled with the ends of worms and an oozy smell, nor yet a dry, bare, sandy hole with nothing in it to sit down on or to eat: it was a hobbit-hole, and that 
means comfort. 
It had a perfectly round door like a porthole, painted green, with a shiny yellow brass knob in the exact middle. The door opened on to a tube-shaped hall like a tunnel: a very comfortable tunnel without smoke, with panelled walls, and floors tiled and carpeted, provided with polished chairs, and lots and lots of pegs for hats and coats - the hobbit was fond of visitors. The tunnel wound on and on, going fairly but not quite straight into the side of the hill - The Hill, as all the people for many miles round called it - and many little round doors opened out of it, first on one side and then on another. 
No going upstairs for the hobbit: bedrooms, bathrooms, cellars, pantries (lots of these), wardrobes (he had whole 
rooms devoted to clothes), kitchens, dining-rooms, all were on the same floor, and indeed on the same passage. 
The best rooms were all on the lefthand side (going in), for these were the only ones to have windows, deep-set round windows looking over his garden, and meadows beyond, sloping down to the river. 

Соберите BPE-словарь размером не более 400 токенов для него. Можно написать собственный скрипт или использовать готовые, но если используете готовые, я попрошу прокомментировать их работу.

In [1]:
# взято с https://github.com/DolbyUUU/byte_pair_encoding_BPE_subword_tokenization_implementation_python/blob/main/BPE.py

# install and import libraries
from collections import Counter, defaultdict
import re

class BPE():

    def __init__(self, corpus, vocab_size):
        """Initialize BPE tokenizer."""
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}
        
    
    # функция для предварительного разбиения текста на слова (использовала простой токенизатор, который был в одной из старых домашних задач)
    def pre_tokenize(self, text):
        processed_text = re.sub(r'[^-^\w\s]', '', text)
        return re.sub(r'-(?!\w)|(?<!\w)-', '', processed_text).split()
            
    # тренируем токенизатор
    def train(self):

        # предварительно разбиваем текст на слова, считаем частотность для каждого слова текста, записываем в словарь
        for text in self.corpus:
            tokens = self.pre_tokenize(text)
            for word in tokens:
                self.word_freqs[word] += 1

        # составляем базовый "алфавит" из всех символов корпуса (сначала в множестве, потом превращаем в список и сортируем)
        alphabet = set()
        for word in self.word_freqs.keys():
            alphabet.update(list(word))
        alphabet = sorted(alphabet)

        # добавляем специальный токен </w> в начале "алфавита"
        vocab = ["</w>"] + alphabet.copy()

        # разбиваем каждое слово на отдельные символы, записываем в словарь
        self.splits = {word: list(word) for word in self.word_freqs.keys()}

        # объединяем самую частотную пару символов; шаг повторяется, пока не  будет достигнут необходимый объем словаря токенов (или пока pair_freqs не будет равен 0)
        while len(vocab) < self.vocab_size:

            # узнаем и записываем частотность каждой пары 
            pair_freqs = self.compute_pair_freqs()
            # прерывается, если словарь pair_freqs пуст, потому что число запрошенных токенов превышает кол-во слов в тексте (больше нет вариантов для объединения)
            if len(pair_freqs) == 0: 
                break
            # находим самую частотную пару
            best_pair = ""
            max_freq = None
            for pair, freq in pair_freqs.items():
                if max_freq is None or max_freq < freq:
                    best_pair = pair
                    max_freq = freq

            # объединяем самую частотную пару, добавляем в "алфавит"
            self.splits = self.merge_pair(*best_pair)
            self.merges[best_pair] = best_pair[0] + best_pair[1]
            vocab.append(best_pair[0] + best_pair[1])
        return self.merges


    def compute_pair_freqs(self):
        # считает частотность пар символов

        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1: 
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs


    def merge_pair(self, a, b):
        # объединяет пару символов (в т.ч. символы, состоящие из объединенных ранее пар)

        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    # перезаписываем в список символов, на которые разбито слово, объединенную пару с-в вместо этих с-в по отдельности
                    split = split[:i] + [a + b] + split[i + 2 :]
                else:
                    i += 1
            self.splits[word] = split
        return self.splits


    def tokenize(self, text):
        # токенизируем предложенный текст

        tokenized_text = self.pre_tokenize(text)
        splits_text = [list(word) for word in tokenized_text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result

In [2]:
text = ''' In a hole in the ground there lived a hobbit. Not a nasty, dirty, wet hole, filled with the ends of worms and an oozy smell,
nor yet a dry, bare, sandy hole with nothing in it to sit down on or to eat: it was a hobbit-hole, and that means comfort.
It had a perfectly round door like a porthole, painted green, with a shiny yellow brass knob in the exact middle.
The door opened on to a tube-shaped hall like a tunnel: a very comfortable tunnel without smoke, with panelled walls,
and floors tiled and carpeted, provided with polished chairs, and lots and lots of pegs for hats and coats - the hobbit was fond of visitors.
The tunnel wound on and on, going fairly but not quite straight into the side of the hill - The Hill, as all the people for many miles round called it -
and many little round doors opened out of it, first on one side and then on another.
No going upstairs for the hobbit: bedrooms, bathrooms, cellars, pantries (lots of these), wardrobes (he had whole
rooms devoted to clothes), kitchens, dining-rooms, all were on the same floor, and indeed on the same passage.
The best rooms were all on the lefthand side (going in), for these were the only ones to have windows, deep-set round windows looking over his garden,
and meadows beyond, sloping down to the river.'''

bpe_tokenizer = BPE(corpus=[text], vocab_size=100)
bpe_tokenizer.train()
tokens = bpe_tokenizer.tokenize(text)
print(tokens)

['I', 'n', 'a', 'hole', 'in', 'the', 'g', 'round', 'the', 'r', 'e', 'li', 'v', 'ed', 'a', 'hobbit', 'N', 'ot', 'a', 'n', 'as', 't', 'y', 'd', 'ir', 't', 'y', 'w', 'et', 'hole', 'f', 'i', 'l', 'led', 'with', 'the', 'e', 'nd', 's', 'of', 'w', 'or', 'm', 's', 'and', 'an', 'o', 'o', 'z', 'y', 's', 'me', 'll', 'n', 'or', 'y', 'et', 'a', 'd', 'r', 'y', 'b', 'ar', 'e', 's', 'and', 'y', 'hole', 'with', 'n', 'o', 'th', 'ing', 'in', 'it', 'to', 's', 'it', 'dow', 'n', 'on', 'or', 'to', 'e', 'a', 't', 'it', 'w', 'as', 'a', 'hobbit', '-', 'hole', 'and', 'th', 'a', 't', 'me', 'an', 's', 'c', 'om', 'for', 't', 'I', 't', 'ha', 'd', 'a', 'p', 'er', 'f', 'e', 'c', 't', 'ly', 'round', 'door', 'li', 'ke', 'a', 'p', 'or', 'th', 'o', 'le', 'p', 'a', 'in', 't', 'ed', 'g', 'r', 'e', 'e', 'n', 'with', 'a', 's', 'h', 'in', 'y', 'y', 'e', 'll', 'ow', 'b', 'r', 'as', 's', 'k', 'n', 'o', 'b', 'in', 'the', 'e', 'x', 'a', 'c', 't', 'm', 'i', 'd', 'd', 'le', 'The', 'door', 'o', 'pe', 'n', 'ed', 'on', 'to', 'a', 'tu',

Посчитайте расстояние Левенштейна для пар слов:

программирование - лингвистика
levenshtein - einstein
Можно вручную, а можно с помощью скрипта, но в таком случае опять будьте готовы комментировать его работу.

In [ ]:
# Левенштейна считала руками

     п  р  о  г  р  а  м  м  и  р  о  в  а  н  и  е
  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16
л 1  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16
и 2  2  2  3  4  5  6  7  8  8  9  10 11 12 13 14 15
н 3  3  3  3  4  5  6  7  8  9  9  10 11 12 12 13 14
г 4  4  4  4  3  4  5  6  7  8  9  10 11 12 13 13 14
в 5  5  5  5  4  4  5  6  7  8  9  10 10 11 12 13 14
и 6  6  6  6  5  5  5  6  7  7  8  9  10 11 12 12 13
с 7  7  7  7  6  6  6  6  7  8  8  9  10 11 12 13 13
т 8  8  8  8  7  7  7  7  7  8  9  9  10 11 12 13 14
и 9  9  9  9  8  8  8  8  8  7  8  9  10 11 12 12 13
к 10 10 10 10 9  9  9  9  9  8  8  9  10 11 12 13 13
а 11 11 11 11 10 10 9  10 10 9  9  9  10 10 11 12 13

answer: 13

     l  e  v  e  n  s  h  h  e  i  n
  0  1  2  3  4  5  6  7  8  9  10 11
e 1  1  1  2  3  4  5  6  7  8  9  10
i 2  2  2  2  3  4  5  6  7  8  9  10
n 3  3  3  3  3  3  4  5  6  7  8   9
s 4  4  4  4  4  4  3  4  5  6  7   8
t 5  5  5  5  5  5  4  4  4  5  6   7
e 6  6  5  6  5  6  5  5  5  4  5   6
i 7  7  6  6  6  6  6  6  6  5  4   5
n 8  8  7  7  7  6  7  7  7  6  5   4

answer: 4

Напишите следующие регулярные выражения:

Для поиска электронной почты. Считаем, что адрес может содержать только латинские буквы, цифры, нижнее подчеркивание и дефис (ну и собачку с точками).

Для поиска пути файла в операционной системе UNIX или Windows (посмотрите: у них различающиеся стандарты). Регулярному выражению должно быть все равно, какая операционная система. Путь файла может быть абсолютным или относительным, может начинаться на букву диска. Регулярное выражение должно содержать две группы, в одной должно отлавливаться имя файла, а в другой его расширение. (Считаем, что у файла обязательно есть расширение).

Дано предложение:

 ```
 #[[Time: Обычно"обычно:#frequentative_adverbs_adj:FREQUENTATIVE"] [Experiencer_Metaphoric: бюджет"бюджет:бюджет:BUDGET"] [[ко"к:#preposition:PREPOSITION"] [OrderInTimeAndSpace: второму"второй:TWO_ORDINAL"] Object_Situation: чтению "чтение:READING_OF_THE_DRAFT_LAW"] Predicate: готовится"готовить:готовить:PREPAREDNESS" [[DegreeApproximative: непосредственно"непосредственный:DIRECT_OBLIQUE"] [в"в_Prepositional:#preposition:PREPOSITION"] Locative: Думе"дума:дума:DUMA"]#: [[Agent: депутаты"депутат:депутат:DEPUTY"] Specification_Clause: корректируют"корректировать:корректировать:TO_CORRECT" [[Agent: правительственные"правительство:правительство:GOVERNMENT"] Object_Situation: планы"план:план:SCHEDULE_FOR_ACTIVITY"]]].
 ```
В этом предложении каждое слово размечено семантической ролью, лексическим классом и семантическим классом. Роль - то, что идет сразу после квадратной скобки до :, потом идет само слово, потом его лемма, а через двоеточие идут лекс. класс и сем. класс (заглавными буквами). Напишите регулярное выражение, которое будет находить 1) сем. роль 2) словоформу 3) лемму 4) лексический класс 5) семантический класс в пяти группах.

In [4]:
# for email

import re

email = 'veridi-grana@yandex.ru viridi.grana@gmail.com'
re_email = r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)'
pattern_email = re.compile(re_email)
re.findall(pattern_email, email)

['veridi-grana@yandex.ru', 'viridi.grana@gmail.com']

In [5]:
# for path (checking for UNIX)

path = '/home/user/docs/Letter.txt'
re_path = r'[\/\\]([^\/\|:*?<>#%()][a-zA-Zа-яА-Я0-9]+).([a-z]+)$'
pattern_path = re.compile(re_path)
re.findall(pattern_path, path)

[('Letter', 'txt')]

In [6]:
# for path (checking for Windows)

path = 'C:\admin\docs\Letter.txt'
re.findall(pattern_path, path)

[('Letter', 'txt')]

In [8]:
# for tagging

# не вполне понятно, писать ли отдельно regex для предлогов: они отличаются по структуре. Также не совcем понимаю, почему у некоторых словоформ 
# как будто нет лексического класса, после словоформы и леммы сразу следует семантический класс.

ling = '''[[Time: Обычно"обычно:#frequentative_adverbs_adj:FREQUENTATIVE"] [Experiencer_Metaphoric: бюджет"бюджет:бюджет:BUDGET"] 
[[ко"к:#preposition:PREPOSITION"] [OrderInTimeAndSpace: второму"второй:TWO_ORDINAL"] Object_Situation: чтению "чтение:READING_OF_THE_DRAFT_LAW"] 
Predicate: готовится"готовить:готовить:PREPAREDNESS" [[DegreeApproximative: непосредственно"непосредственный:DIRECT_OBLIQUE"] 
[в"в_Prepositional:#preposition:PREPOSITION"] Locative: Думе"дума:дума:DUMA"]#: [[Agent: депутаты"депутат:депутат:DEPUTY"] 
Specification_Clause: корректируют"корректировать:корректировать:TO_CORRECT" [[Agent: правительственные"правительство:правительство:GOVERNMENT"] 
Object_Situation: планы"план:план:SCHEDULE_FOR_ACTIVITY"]]].'''

re_ling = r'([A-Z]{1}[A-Za-z_]+):.([А-Яа-я]+).?\"([а-я]+):([а-я]+)?:?([a-zA-Z_:#]+)\"'
# pattern_ling = re.compile(re_ling)
# re.findall(pattern_ling, ling)

for elem in re.finditer(re_ling, ling):
  print(elem.group(1), elem.group(2), elem.group(3), elem.group(4), elem.group(5), sep=', ')

Time, Обычно, обычно, None, #frequentative_adverbs_adj:FREQUENTATIVE
Experiencer_Metaphoric, бюджет, бюджет, бюджет, BUDGET
OrderInTimeAndSpace, второму, второй, None, TWO_ORDINAL
Object_Situation, чтению, чтение, None, READING_OF_THE_DRAFT_LAW
Predicate, готовится, готовить, готовить, PREPAREDNESS
DegreeApproximative, непосредственно, непосредственный, None, DIRECT_OBLIQUE
Locative, Думе, дума, дума, DUMA
Agent, депутаты, депутат, депутат, DEPUTY
Specification_Clause, корректируют, корректировать, корректировать, TO_CORRECT
Agent, правительственные, правительство, правительство, GOVERNMENT
Object_Situation, планы, план, план, SCHEDULE_FOR_ACTIVITY
